## Лекция 3  NER

### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)

In [116]:
test_text = '''Добрый день!

Пожалуйста, уточните, на данный момент для посещения Камчатки требуется ли предоставление отрицательных тестов на COVID19 или какие-либо другие документов, помимо паспорта гражданина РФ для прибывающих из Москвы регулярным рейсом?

Нужно ли оформлять какие-либо пропуски для пребывания на тер-рии Камчатки? Целью поездки явлеятся туризм, проживание заранее оплачивается, экскурсионные услуги будут организованы непосредственно через отель. Есть ли другие ограничения или меры, о которых необходимо знать туристам, которые отправляются на Камчатку (11-16 августа)


Заранее спасибо

Елена Мягкова
'''

In [117]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsNERTagger,

    Doc
)

segmenter = Segmenter()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

In [118]:
import re
from razdel import tokenize, sentenize
from nltk.corpus import stopwords
stops = stopwords.words("russian")
stops.extend(['здравствовать', 'добрый', 'день', 'спасибо', 'пожалуйста'])

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

def tokenizer(text_data):
    tokens = [_.text for _ in list(tokenize(str(text_data).lower())) if not re.search('[^а-яА-ЯёЁa-zA-z]', _.text)]
    return " ".join(tokens)

def lemmatizer(tokens):
    lem_text = []
    for word in tokens.split(" "):
        lem = morph.parse(word)[0].normal_form
        if lem not in stops:
            lem_text.append(lem)
    return ' '.join(lem_text)

In [119]:
from tqdm.auto import tqdm
tqdm.pandas()

In [120]:
def preprocess_with_natasha(text: str) -> str:
    #text = text.replace('\n', ' ')
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        text = text.replace(span.text, "")
    return lemmatizer(tokenizer(text)) #re.sub(r"([^\w\s])", r" \1 ", new_text) 

In [121]:
preprocess_with_natasha(test_text)

'уточнить данный момент посещение требоваться предоставление отрицательный тест covid документ помимо паспорт гражданин прибывать регулярный рейс нужно оформлять пропуск пребывание цель поездка явлеиться туризм проживание заранее оплачиваться экскурсионный услуга организовать непосредственно отель ограничение мера который необходимый знать турист который отправляться август заранее'

In [122]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2020-09-30 02:11:13.293 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
I0930 02:11:13.293884 14332 download.py:117] Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2020-09-30 02:11:15.866 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
I0930 02:11:15.866795 14332 download.py:117] Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
2020-09-30 02:11:16.276 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from C:\Users\User\.deeppavlov\models\ner_rus_bert\tag.dict]
I0930 02:11:16.276700 14332 simple_vocab.py:112] [loading vocabulary from C:\Users\User\.dee

2020-09-30 02:11:48.873 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from C:\Users\User\.deeppavlov\models\ner_rus_bert\model]
I0930 02:11:48.873064 14332 tf_model.py:52] [loading model from C:\Users\User\.deeppavlov\models\ner_rus_bert\model]


In [123]:
def preprocess_with_deepmipt(text: str) -> str:
    new_text = []
    sentences_list = [_.text for _ in list(sentenize(text))]
    res = ner_model(sentences_list)
    for sentence, entities in zip(res[0], res[1]):
        for token, entity in zip(sentence, entities):
            if entity == "O":
                new_text.append(token)
    return lemmatizer(tokenizer(' '.join(new_text))) #re.sub(r"\s+", " ", ' '.join(new_text))

In [124]:
preprocess_with_deepmipt(test_text)

'уточнить данный момент посещение требоваться предоставление отрицательный тест covid либо документ помимо паспорт гражданин прибывать регулярный рейс нужно оформлять либо пропуск пребывание тереть рии цель поездка явлеиться туризм проживание заранее оплачиваться экскурсионный услуга организовать непосредственно отель ограничение мера который необходимый знать турист который отправляться август заранее'

### __Задача 2__:    
На предыдущем занятии вы реализовывали функции поиска ближайших ответов на запросы через TF-IDF и BM25. 
Сравните качество нахождения верного ответа для обоих методов в трех случаях:
- с функцией ```preprocess_with_natasha```
- с функцией ```preprocess_with_deepmipt```
- без препроцессинга

Для измерения качества используйте метрику accuracy. Считаем, что ответ верный, если он входит в топ-1.

In [125]:
import pandas as pd
import numpy as np

Готовлю данные

In [126]:
queries_data = pd.read_excel("queries_base.xlsx").fillna('none')

In [128]:
queries_data["natasha"] = queries_data["Текст вопроса"].progress_apply(preprocess_with_natasha)
queries_data.to_csv('queries_data_natasha.tsv', sep='\t')

In [129]:
queries_data["deeppavlov"] = queries_data["Текст вопроса"].progress_apply(preprocess_with_deepmipt)
queries_data.to_csv('queries_data_natasha_deeppavlov.tsv', sep='\t')

In [130]:
train = queries_data[["Текст вопроса", "natasha", "deeppavlov", "Номер связки\n"]][:int(len(queries_data)*0.7)]
train = train.rename(columns = {"Текст вопроса": "question", "Номер связки\n": "index"}, inplace = False)

test = queries_data[["Текст вопроса", "natasha", "deeppavlov", "Номер связки\n"]][int(len(queries_data)*0.7):]
test = test.rename(columns = {"Текст вопроса": "question", "Номер связки\n": "index"}, inplace = False)

In [131]:
train.tail()

,question,natasha,deeppavlov,index
1604,Здравствуйте меня зовут Александр Архипов. Я г...,звать гражданин работать филиал иностранный ор...,звать гражданин работать филиал иностранный ор...,308
1605,Результат Пцр мазка Мещанинова Константина Вал...,результат мазка,результат пцр мазка,6
1606,Я являюсь сотрудником министерства гражданской...,являться сотрудник министерство гражданский об...,являться сотрудник министерство гражданский об...,1
1607,"Здравствуйте!Мне нужно уточнить по вопросу, ка...",нужно уточнить вопрос касаться возвращение пое...,нужно уточнить вопрос касаться возвращение пое...,308
1608,"Здравствуйте!\nМне нужно уточнить по вопросу, ...",нужно уточнить вопрос касаться возвращение пое...,нужно уточнить вопрос касаться возвращение пое...,308


In [132]:
answers_data = pd.read_excel("answers_base.xlsx")

question_df = pd.DataFrame(columns=["index", "question", "natasha", "deeppavlov"])
for question_chunk, answer_n in tqdm(answers_data[["Текст вопросов", "Номер связки"]].values):
    questions = question_chunk.split('\n')
    for q in questions:
        question_df = question_df.append({"index": answer_n,
                            "question": q,
                            "natasha": preprocess_with_natasha(q),
                            "deeppavlov": preprocess_with_deepmipt(q)},
                           ignore_index=True)

question_df.head()

,index,question,natasha,deeppavlov
0,57,У ребенка в школе продлили каникулы. Могу ли я...,ребёнок школа продлить каникулы мочь взять бол...,ребёнок школа продлить каникулы мочь взять бол...
1,57,Больничный лист?,больничный лист,больничный лист
2,57,"Есть ли компенсация, в случае если есть разниц...",компенсация случай разница оплата больничный з...,компенсация случай разница оплата больничный з...
3,57,как оплачивается больничный при коронавирусе?,оплачиваться больничный коронавирус,оплачиваться больничный коронавирус
4,57,"Я контактный, дадут ли больничный?",контактный дать больничный,контактный дать больничный


In [133]:
train = train.append(question_df, sort=False, ignore_index = True)
train.tail()

,question,natasha,deeppavlov,index
2385,"Где можно ознакомиться с нормативными, методич...",ознакомиться нормативный методический документ...,ознакомиться нормативный методический документ...,45
2386,Правительство явно что-то скрывает о ситуации ...,правительство явно скрывать ситуация коронавирус,правительство явно скрывать ситуация коронавирус,21
2387,Как узнать реальное количество заболевших?,узнать реальный количество заболеть,узнать реальный количество заболеть,21
2388,"Какая эпидемиологическая ситуация в мире, Росс...",эпидемиологический ситуация мир мыть регион,эпидемиологический ситуация мир мыть регион,21
2389,В каких странах неблагополучная эпидситуация?,страна неблагополучный эпидситуация,страна неблагополучный эпидситуация,21


Собираю bm25 в одну функцию от обучающей выборки, тестовой выборки и названия столбца с нужным препроцессингом

In [134]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from math import log

In [135]:
def bm25(train, test, col_name):
    count_vectorizer = CountVectorizer(analyzer=str.split)
    count_matrix = count_vectorizer.fit_transform(train[col_name])
    print(count_matrix.shape)

    k = 2.0
    b = 0.75

    length = count_matrix.sum(axis=1)
    avgdl = length.mean()
    tf_matrix = (count_matrix * (k+1)) / (count_matrix + k * (1 - b + b * (length/avgdl) ))
    print(tf_matrix.shape)

    n = (tf_matrix != 0).sum(axis=0)
    N = count_matrix.shape[0]
    def idf(i):
        res = log( (N - n[0, i] + 0.5) / (n[0, i] + 0.5) )
        return res

    idf_vector = np.array([idf(i) for i in range(count_matrix.shape[1])])
    print(idf_vector.shape)
    
    def bm25_search(q):
        q_vec = count_vectorizer.transform([q]).toarray()
        mask = q_vec * idf_vector
        res = np.dot(tf_matrix, mask.T)
        index = np.argmax(res)
        return train["index"][index]
    
    return test[col_name].progress_apply(bm25_search)

Собираю TF-IDF в одну функцию от от обучающей выборки, тестовой выборки и названия столбца с нужным препроцессингом

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(train, test, col_name):
    vectorizer = TfidfVectorizer(analyzer=str.split)
    X = vectorizer.fit_transform(train[col_name])
    #X.shape
    
    def tfidf_search(querry_lem):
        q_vec = vectorizer.transform([querry_lem]).toarray()
        res = np.dot(X.toarray(), q_vec.T)
        index = np.argmax(res)
        return train["index"][index]
    
    return test[col_name].progress_apply(tfidf_search)

Делаю таблицу для результатов подсчета accuracy

In [138]:
col_names = ["question", "natasha", "deeppavlov"]

res_df = pd.DataFrame(index=["tf-idf", "bm25"], columns=col_names)
res_df['only_lem'] = [0.4927536231884058, 0.527536231884058]
res_df

,question,natasha,deeppavlov,only_lem
tf-idf,NaN,NaN,NaN,0.492754
bm25,NaN,NaN,NaN,0.527536


Заполняю все ячейки, применяя два разных поиска к данным с термя разными препроцессингами

In [139]:
for col_name in col_names:
    test["pred_"+col_name+"_bm25"] = bm25(train, test, col_name)
    test["pred_"+col_name+"_tfidf"] = tfidf(train, test, col_name)
    
    res_df[col_name]["tf-idf"] = ((test["pred_"+col_name+"_tfidf"] == test['index']).sum() / len(test))
    res_df[col_name]["bm25"] = ((test["pred_"+col_name+"_bm25"] == test['index']).sum() / len(test))

(2390, 19476)
(2390, 19476)
(19476,)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(2390, 4749)
(2390, 4749)
(4749,)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(2390, 4712)
(2390, 4712)
(4712,)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Итоговые значения accuracy

In [140]:
res_df

,question,natasha,deeppavlov,only_lem
tf-idf,0.475362,0.45942,0.462319,0.492754
bm25,0.481159,0.514493,0.521739,0.527536


### __Задача 3__:    
Улучшить правила в natasha. Написать правила, которые ловят даты в следующих примерах и пересчитать статистику из Задачи 2:
- Уехал 8-9 ноября в Сочи
- Уезжаю 5 числа                           
- 20го сентября заболел

Пример можно посмотреть тут: https://github.com/natasha/yargy